In [277]:
import numpy as np
from matplotlib import pyplot as plt
import wordcloud
import requests
import re
import os 
from bs4 import BeautifulSoup
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import MultinomialNB
import os
import urllib.parse
from re import search

In [227]:
path = "/home/garima/Desktop/SpicedAcademy/spiced_projects/repos/random-rose-student-code/week_4/Milestone-Project-4/"
os.chdir(path)

In [228]:
url = 'https://www.lyrics.com/artist/'

In [229]:
def get_artist_link(artistname):
    url_artistname = url + artistname
    return url_artistname

In [230]:
artist_list  = ['Ed-Sheeran','Rihanna']

In [231]:
# to get URL for different artists
artist_links = []

for i in range(len(artist_list)):
    artist_links.append(get_artist_link(artist_list[i]))
print(artist_links)

['https://www.lyrics.com/artist/Ed-Sheeran', 'https://www.lyrics.com/artist/Rihanna']


In [232]:
# create dictionary with artist names as keys and links as values
dict_artist = dict( zip(artist_list,artist_links ))
dict_artist

{'Ed-Sheeran': 'https://www.lyrics.com/artist/Ed-Sheeran',
 'Rihanna': 'https://www.lyrics.com/artist/Rihanna'}

In [233]:
dict_artist['Ed-Sheeran']

'https://www.lyrics.com/artist/Ed-Sheeran'

In [234]:
response = requests.get(dict_artist['Ed-Sheeran'])

In [235]:
response.status_code

200

In [236]:
soup_lyrics_1 = BeautifulSoup(response.text,'html.parser')

In [237]:
html_links_1 = soup_lyrics_1.find_all(class_="tal qx")
html_links_1

[<td class="tal qx"><strong><a href="/lyric/36137186/Ed+Sheeran/Lay+It+All+on+Me">Lay It All on Me</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36131004/Ed+Sheeran/Lay+It+All+on+Me">Lay It All on Me</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36370642/Ed+Sheeran/BLOW">BLOW</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36355896/Ed+Sheeran/Beautiful+People">Beautiful People</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36370638/Ed+Sheeran/Best+Part+of+Me">Best Part of Me</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36234915/Ed+Sheeran/Cross+Me">Cross Me</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36156455/Ed+Sheeran/Beast+of+Burden">Beast of Burden</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36221045/Ed+Sheeran/I+Don%27t+Care">I Don't Care</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/36259016/Ed+Sheeran/Beautiful+People">Beautiful People</a></st

In [238]:
song_links_1 = []
for link in html_links_1:
    int_link= link.a['href']
    url_song = "https://www.lyrics.com" + int_link
    song_links_1.append(url_song)
song_links_1 = list(set(song_links_1))
print(song_links_1)

['https://www.lyrics.com/lyric/30818520/Ed+Sheeran/Afire+Love', 'https://www.lyrics.com/lyric/33552984/Ed+Sheeran/You+Need+Me%2C+I+Don%27t+Need+You', 'https://www.lyrics.com/lyric/30285988/Ed+Sheeran/You+Need+Me%2C+I+Don%27t+Need+You', 'https://www.lyrics.com/lyric/36739215/Ed+Sheeran/Own+It', 'https://www.lyrics.com/lyric/32789831/Ed+Sheeran/Photograph', 'https://www.lyrics.com/lyric/24854127/Ed+Sheeran/Small+Bump', 'https://www.lyrics.com/lyric/36137186/Ed+Sheeran/Lay+It+All+on+Me', 'https://www.lyrics.com/lyric-lf/1578469/Ed+Sheeran/Take+Me+Back+To+London+%5BRemix%5D', 'https://www.lyrics.com/lyric/38857058/Ed+Sheeran/Leave+Your+Life', 'https://www.lyrics.com/lyric/31631530/Ed+Sheeran/Don%27t', 'https://www.lyrics.com/lyric/33354381/Ed+Sheeran/Photograph', 'https://www.lyrics.com/lyric/33552975/Ed+Sheeran/This', 'https://www.lyrics.com/lyric/32084465/Ed+Sheeran/Bloodstream', 'https://www.lyrics.com/lyric/25080763/Ed+Sheeran/The+City', 'https://www.lyrics.com/lyric/34840658/Ed+Sheera

In [239]:
song_links_init_1 = [each_string.lower() for each_string in song_links_1]
song_links_init_1

['https://www.lyrics.com/lyric/30818520/ed+sheeran/afire+love',
 'https://www.lyrics.com/lyric/33552984/ed+sheeran/you+need+me%2c+i+don%27t+need+you',
 'https://www.lyrics.com/lyric/30285988/ed+sheeran/you+need+me%2c+i+don%27t+need+you',
 'https://www.lyrics.com/lyric/36739215/ed+sheeran/own+it',
 'https://www.lyrics.com/lyric/32789831/ed+sheeran/photograph',
 'https://www.lyrics.com/lyric/24854127/ed+sheeran/small+bump',
 'https://www.lyrics.com/lyric/36137186/ed+sheeran/lay+it+all+on+me',
 'https://www.lyrics.com/lyric-lf/1578469/ed+sheeran/take+me+back+to+london+%5bremix%5d',
 'https://www.lyrics.com/lyric/38857058/ed+sheeran/leave+your+life',
 'https://www.lyrics.com/lyric/31631530/ed+sheeran/don%27t',
 'https://www.lyrics.com/lyric/33354381/ed+sheeran/photograph',
 'https://www.lyrics.com/lyric/33552975/ed+sheeran/this',
 'https://www.lyrics.com/lyric/32084465/ed+sheeran/bloodstream',
 'https://www.lyrics.com/lyric/25080763/ed+sheeran/the+city',
 'https://www.lyrics.com/lyric/3484

In [240]:
titles_1 = set()
links_list_1 = []

for x in song_links_init_1:   
    y = x.split('/')[-1].replace("+", " ")
    if not search("%5b", y):
        if not search("%28", y):
            y = urllib.parse.unquote(y)
            if y not in titles_1:
                links_list_1.append(x)
                titles_1.add(y)

In [241]:
links_list_1

['https://www.lyrics.com/lyric/30818520/ed+sheeran/afire+love',
 'https://www.lyrics.com/lyric/33552984/ed+sheeran/you+need+me%2c+i+don%27t+need+you',
 'https://www.lyrics.com/lyric/36739215/ed+sheeran/own+it',
 'https://www.lyrics.com/lyric/32789831/ed+sheeran/photograph',
 'https://www.lyrics.com/lyric/24854127/ed+sheeran/small+bump',
 'https://www.lyrics.com/lyric/36137186/ed+sheeran/lay+it+all+on+me',
 'https://www.lyrics.com/lyric/38857058/ed+sheeran/leave+your+life',
 'https://www.lyrics.com/lyric/31631530/ed+sheeran/don%27t',
 'https://www.lyrics.com/lyric/33552975/ed+sheeran/this',
 'https://www.lyrics.com/lyric/32084465/ed+sheeran/bloodstream',
 'https://www.lyrics.com/lyric/25080763/ed+sheeran/the+city',
 'https://www.lyrics.com/lyric/31181170/ed+sheeran/all+about+it',
 'https://www.lyrics.com/lyric/35303606/ed+sheeran/shape+of+you',
 'https://www.lyrics.com/lyric/32347467/ed+sheeran/the+man',
 'https://www.lyrics.com/lyric/34723951/ed+sheeran/perfect+symphony',
 'https://www

In [242]:
titles_1

{'10,000 tears',
 '1000 nights',
 '2step',
 'a team',
 'afire love',
 'afterglow',
 'all about it',
 'all of the stars',
 'amo soltanto te',
 'antisocial',
 'atlantic city',
 'autumn leaves',
 'bad habits',
 'bam bam',
 'barcelona',
 'be like you',
 'be my forever',
 'be my husband',
 'be right now',
 'beast of burden',
 'beautiful people',
 'best part of me',
 'beyond the pale',
 'bibia be ye ye',
 'bloodstream',
 'blow',
 'boa me',
 'brace it',
 'candle in the wind',
 'castle on the hill',
 'chasing cars',
 'city of sin',
 'cold coffee',
 'collide',
 'crocodile',
 'cross me',
 'dans 10 ans',
 'dark times',
 'diary',
 'dive',
 "don't",
 "don't - loyal - no diggity - the next episode - nina",
 'dreams',
 'drown me out',
 'drunk',
 'drunk in love',
 'dumb for you',
 'end game',
 'english rose',
 'eraser',
 'even my dad does sometimes',
 'everything has changed',
 'everything you are',
 'faces',
 'fairytale of new york',
 'fall',
 'family',
 'feels',
 'fire alarms',
 'firefly',
 'first t

In [243]:
from bs4 import SoupStrainer
lyrics_list_1 = []
#for link in lists[:100]:
for link in links_list_1:
    k = requests.get(link)
    urh=BeautifulSoup(k.content, parse_only = SoupStrainer(class_='lyric-body'))
    lyrics_list_1.append(urh.text)

In [244]:
lyrics_list_1

["Things were all good yesterday\nAnd then the devil took your memory\nAnd if you fell to your death today\nI hope that heaven is your resting place\nI heard the doctors put your chest in pain\nBut then that could have been the medicine\nAnd now you’re lying in the bed again\nEither way I’ll cry with the rest of them\n\nAnd my father told me, son\nIt’s not his fault he doesn’t know your face\nAnd you’re not the only one\nAlthough my grandma used to say, he used to say\n\nDarling hold me in your arms the way you did last night\nAnd we'll lie inside, a little while he wrote\nI could look into your eyes until the sun comes up\nAnd we’re wrapped in light, in life, in love\nPut your open lips on mine and slowly let them shut\nFor they’re designed to be together oh\nWith your body next to mine our hearts will beat as one\nAnd we’re set alight, we’re afire love, love, love oh\n\nAnd things were all good yesterday\nThen the devil took your breath away\nAnd now we’re left here in the pain\nBlac

In [245]:
import pandas as pd
lyrics_df_1 = pd.DataFrame(lyrics_list_1)

In [246]:
lyrics_df_1.columns = ["Lyrics"]

In [257]:
lyrics_df_1

,Lyrics
0,Things were all good yesterday\nAnd then the d...
1,"Now I'm in town, break it down, thinking of ma..."
2,"Own it, girl, you just own it\n'Cause your bod..."
3,"Loving can hurt, loving can hurt sometimes\r\n..."
4,"You were just a small bump unborn, in four mon..."
...,...
173,"Yeah, Slumdon Bridge\r\nMr. Red, Catfish\r\nBa..."
174,I keep my hopes and dreams inside of you among...
175,"Ye ne boa me\r\nYe ne boa me, wai\r\nSo me mu ..."
176,"I am a poor, wayfaring stranger\nTraveling thr..."


In [258]:
lyrics_df_1.shape

(178, 1)

In [259]:
len(song_links_init_1)

713

In [260]:
response_2 = requests.get(dict_artist['Rihanna'])

In [261]:
response_2.status_code

200

In [262]:
soup_lyrics_2 = BeautifulSoup(response_2.text,'html.parser')

In [263]:
html_links_2 = soup_lyrics_2.find_all(class_="tal qx")
html_links_2

[<td class="tal qx"><strong><a href="/lyric/35074593/Rihanna/Yeah%2C+I+Said+It">Yeah, I Said It</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/35074564/Rihanna/Love+on+the+Brain">Love on the Brain</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/34667746/Rihanna/Love+on+the+Brain">Love on the Brain</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/33695385/Rihanna/Love+on+the+Brain">Love on the Brain</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/33913852/Rihanna/LOYALTY.+FT.+RIHANNA">LOYALTY. FT. RIHANNA</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/33916084/Rihanna/LOYALTY.">LOYALTY.</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/34661233/Rihanna/LOYALTY.">LOYALTY.</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/33772976/Rihanna/Selfish">Selfish</a></strong></td>,
 <td class="tal qx"><strong><a href="/lyric/33730925/Rihanna/Needed+Me">Needed Me</a></strong></td>,
 <td class="tal

In [264]:
song_links_2 = []
for link in html_links_2:
    int_link= link.a['href']
    url_song = "https://www.lyrics.com" + int_link
    song_links_2.append(url_song)
#song_links_2 = list(set(song_links_2))
print(song_links_2)

['https://www.lyrics.com/lyric/35074593/Rihanna/Yeah%2C+I+Said+It', 'https://www.lyrics.com/lyric/35074564/Rihanna/Love+on+the+Brain', 'https://www.lyrics.com/lyric/34667746/Rihanna/Love+on+the+Brain', 'https://www.lyrics.com/lyric/33695385/Rihanna/Love+on+the+Brain', 'https://www.lyrics.com/lyric/33913852/Rihanna/LOYALTY.+FT.+RIHANNA', 'https://www.lyrics.com/lyric/33916084/Rihanna/LOYALTY.', 'https://www.lyrics.com/lyric/34661233/Rihanna/LOYALTY.', 'https://www.lyrics.com/lyric/33772976/Rihanna/Selfish', 'https://www.lyrics.com/lyric/33730925/Rihanna/Needed+Me', 'https://www.lyrics.com/lyric/33730904/Rihanna/Sex+With+Me', 'https://www.lyrics.com/lyric/33805620/Rihanna/Needed+Me', 'https://www.lyrics.com/lyric/33805599/Rihanna/Sex+With+Me', 'https://www.lyrics.com/lyric/33805593/Rihanna/Famous', 'https://www.lyrics.com/lyric/34611173/Rihanna/LOYALTY.', 'https://www.lyrics.com/lyric/34606697/Rihanna/Lemon', 'https://www.lyrics.com/lyric/34987854/Rihanna/Lemon', 'https://www.lyrics.com/

In [265]:
song_links_init_2 = [each_string.lower() for each_string in song_links_2]
song_links_init_2

['https://www.lyrics.com/lyric/35074593/rihanna/yeah%2c+i+said+it',
 'https://www.lyrics.com/lyric/35074564/rihanna/love+on+the+brain',
 'https://www.lyrics.com/lyric/34667746/rihanna/love+on+the+brain',
 'https://www.lyrics.com/lyric/33695385/rihanna/love+on+the+brain',
 'https://www.lyrics.com/lyric/33913852/rihanna/loyalty.+ft.+rihanna',
 'https://www.lyrics.com/lyric/33916084/rihanna/loyalty.',
 'https://www.lyrics.com/lyric/34661233/rihanna/loyalty.',
 'https://www.lyrics.com/lyric/33772976/rihanna/selfish',
 'https://www.lyrics.com/lyric/33730925/rihanna/needed+me',
 'https://www.lyrics.com/lyric/33730904/rihanna/sex+with+me',
 'https://www.lyrics.com/lyric/33805620/rihanna/needed+me',
 'https://www.lyrics.com/lyric/33805599/rihanna/sex+with+me',
 'https://www.lyrics.com/lyric/33805593/rihanna/famous',
 'https://www.lyrics.com/lyric/34611173/rihanna/loyalty.',
 'https://www.lyrics.com/lyric/34606697/rihanna/lemon',
 'https://www.lyrics.com/lyric/34987854/rihanna/lemon',
 'https:/

In [266]:
len(song_links_init_2)

2064

In [267]:
titles_2 = set()
links_list_2 = []

for x in song_links_init_2:   
    y = x.split('/')[-1].replace("+", " ")
    y = y.replace("%3f", "")
    if not search("%5b", y):
        if not search("%28", y):
            if not search(" pt", y):
                y = urllib.parse.unquote(y)
                if y not in titles_2:
                    links_list_2.append(x)
                    titles_2.add(y)


In [268]:
links_list_2

['https://www.lyrics.com/lyric/35074593/rihanna/yeah%2c+i+said+it',
 'https://www.lyrics.com/lyric/35074564/rihanna/love+on+the+brain',
 'https://www.lyrics.com/lyric/33913852/rihanna/loyalty.+ft.+rihanna',
 'https://www.lyrics.com/lyric/33916084/rihanna/loyalty.',
 'https://www.lyrics.com/lyric/33772976/rihanna/selfish',
 'https://www.lyrics.com/lyric/33730925/rihanna/needed+me',
 'https://www.lyrics.com/lyric/33730904/rihanna/sex+with+me',
 'https://www.lyrics.com/lyric/33805593/rihanna/famous',
 'https://www.lyrics.com/lyric/34606697/rihanna/lemon',
 'https://www.lyrics.com/lyric/33880055/rihanna/nothing+is+promised',
 'https://www.lyrics.com/lyric/32569378/rihanna/consideration',
 'https://www.lyrics.com/lyric/32569377/rihanna/james+joint',
 'https://www.lyrics.com/lyric/32569376/rihanna/kiss+it+better',
 'https://www.lyrics.com/lyric/32569375/rihanna/work',
 'https://www.lyrics.com/lyric/32569374/rihanna/desperado',
 'https://www.lyrics.com/lyric/32569373/rihanna/woo',
 'https://w

In [269]:
len(links_list_2)

179

In [270]:
titles_2

{'a child is born',
 'a girl like me',
 'a million miles away',
 'american oxygen',
 'answer',
 'as real as you and me',
 "baby i'm back",
 'believe it',
 'birthday cake',
 'bitch better have my money',
 'boom boom',
 'break it off',
 "breakin' dishes",
 'bubble pop',
 'california king bed',
 "can't remember to forget you",
 'cinco minutos',
 'close to you',
 'cold case love',
 'complicated',
 'consideration',
 'coulda been the one',
 'crazy little thing called love',
 'cry',
 'dancing in the dark',
 'dem haters',
 'desperado',
 'diamonds',
 'disturbia',
 'do ya thang',
 "don't stop the music",
 'drunk on love',
 'electricity',
 'fading',
 'famous',
 'farewell',
 'feel the adrenaline',
 'final goodbye',
 'fire bomb',
 'first time',
 'fly',
 'fool in love',
 'fourfiveseconds',
 'fresh off the runway',
 'g4l',
 'get it over with',
 'girl like me',
 'good girl gone bad',
 'goodnight gotham',
 'half of me',
 'happy',
 'happy hour',
 'hard',
 'hate that i love you',
 "hatin' on the club",
 

In [271]:
len(titles_2)

179

In [272]:
from bs4 import SoupStrainer
lyrics_list_2 = []
#for link in lists[:100]:
for link in links_list_2:
    k = requests.get(link)
    urh=BeautifulSoup(k.content, parse_only = SoupStrainer(class_='lyric-body'))
    lyrics_list_2.append(urh.text)

In [273]:
lyrics_list_2

["Yeah, yeah\r\nYeah, yeah\r\n\r\nI ain't tryin' to think about it, no\r\nYeah, I said it, boy, get up inside it, uh\r\nI want you to homicide it\r\nGoing slow and I want you to pop it uh\r\nAnd I think I kinda like ya\r\nUp against the wall, we don't need a title uh\r\n\r\nYeah, I said it\r\nYeah, I said it, babe\r\nYeah, I said it, f*ck a title uh\r\n\r\nBoy I always like to show\r\nGet a little bit, come a little close, no\r\nTake it home on your camera phone\r\nGet a little bad, nigga, watch me blow it down\r\n\r\nYeah, I said it\r\nYeah, I said it, babe\r\nYeah, I said it (ooh yeah)\r\n\r\nYeah, I said it\r\nYeah, I said it, babe\r\nYeah, I said it (ooh yeah)\r\n \r\nYou could be rough, boy, but if you won't\r\nGive me some love, boy, give it to me till the morn'\r\n\r\nYeah, I said it\r\nYeah, I said it, babe\r\nYeah, I said it uh\r\n\r\nYeah, I said it\r\nYeah, I said it, babe\r\nYeah, I said it\r\n\r\nOh yeah\r\nYeah yeah\r\nYeah yeah yeah yeah yeah",
 "And you got me like, oh\

In [274]:
lyrics_df_2 = pd.DataFrame(lyrics_list_2)

In [275]:
lyrics_df_2.columns = ["Lyrics"]

In [276]:
lyrics_df_2.shape

(179, 1)

In [278]:
CORPUS = lyrics_list_1 + lyrics_list_2

In [279]:
len(CORPUS)

357

In [280]:
# let's lower case everything

CORPUS = [s.lower() for s in CORPUS]

In [281]:
CORPUS

["things were all good yesterday\nand then the devil took your memory\nand if you fell to your death today\ni hope that heaven is your resting place\ni heard the doctors put your chest in pain\nbut then that could have been the medicine\nand now you’re lying in the bed again\neither way i’ll cry with the rest of them\n\nand my father told me, son\nit’s not his fault he doesn’t know your face\nand you’re not the only one\nalthough my grandma used to say, he used to say\n\ndarling hold me in your arms the way you did last night\nand we'll lie inside, a little while he wrote\ni could look into your eyes until the sun comes up\nand we’re wrapped in light, in life, in love\nput your open lips on mine and slowly let them shut\nfor they’re designed to be together oh\nwith your body next to mine our hearts will beat as one\nand we’re set alight, we’re afire love, love, love oh\n\nand things were all good yesterday\nthen the devil took your breath away\nand now we’re left here in the pain\nblac

In [282]:
#tokenize and lemmaztize
from nltk.tokenize import WhitespaceTokenizer

CLEAN_CORPUS = []

#tokenizer = TreebankWordTokenizer()
tokenizer = WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()


for doc in CORPUS:
    tokens = tokenizer.tokenize(text=doc)
    clean_doc = " ".join(lemmatizer.lemmatize(token) for token in tokens)
    CLEAN_CORPUS.append(clean_doc)

In [283]:
CLEAN_CORPUS

["thing were all good yesterday and then the devil took your memory and if you fell to your death today i hope that heaven is your resting place i heard the doctor put your chest in pain but then that could have been the medicine and now you’re lying in the bed again either way i’ll cry with the rest of them and my father told me, son it’s not his fault he doesn’t know your face and you’re not the only one although my grandma used to say, he used to say darling hold me in your arm the way you did last night and we'll lie inside, a little while he wrote i could look into your eye until the sun come up and we’re wrapped in light, in life, in love put your open lip on mine and slowly let them shut for they’re designed to be together oh with your body next to mine our heart will beat a one and we’re set alight, we’re afire love, love, love oh and thing were all good yesterday then the devil took your breath away and now we’re left here in the pain black suit, black tie standin' in the rain

In [284]:
# create labels for classification

LABELS = ['ed sheeran'] * 178 + ['rihanna'] * 179
LABELS

['ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheeran',
 'ed sheer

### Vectorize Text

In [285]:
vectorizer = CountVectorizer() # instanciation

In [286]:
vectors = vectorizer.fit_transform(CLEAN_CORPUS)

In [287]:
vectors

<357x6397 sparse matrix of type '<class 'numpy.int64'>'
	with 41564 stored elements in Compressed Sparse Row format>

In [288]:
vectors.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [289]:
vectorizer.get_feature_names()

/home/garima/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '01',
 '03',
 '06',
 '10',
 '12',
 '143',
 '15',
 '16',
 '17',
 '1738',
 '18',
 '20',
 '2001',
 '2010',
 '2015',
 '21',
 '22',
 '23',
 '24',
 '25',
 '27',
 '29th',
 '2pac',
 '30',
 '31',
 '50',
 '68',
 '70',
 '80',
 '90',
 '911',
 'a14',
 'aaaaaah',
 'aan',
 'abajo',
 'abandoning',
 'able',
 'abonnés',
 'abort',
 'abou',
 'about',
 'above',
 'abrantee',
 'absence',
 'absinthe',
 'abuse',
 'abused',
 'acabar',
 'accept',
 'access',
 'accident',
 'acción',
 'accommodation',
 'accompagnato',
 'according',
 'account',
 'accumulerais',
 'accuse',
 'ace',
 'ache',
 'achieve',
 'achieved',
 'achieving',
 'aching',
 'acid',
 'acidic',
 'acobardarme',
 'acquired',
 'acquis',
 'acre',
 'across',
 'act',
 'actin',
 'acting',
 'action',
 'actor',
 'actress',
 'actually',
 'acá',
 'ad',
 'add',
 'addict',
 'addicted',
 'addiction',
 'adding',
 'addition',
 'addy',
 'admiring',
 'admit',
 'admittin',
 'adn',
 'adolescence',
 'adore',
 'adores',
 'adoro',
 'adrenaline',
 'adrien',
 'adultère

In [290]:
# To put the vectorized lyrics in a pandas dataframe with the labels

pd.DataFrame(vectors.todense(), columns=vectorizer.get_feature_names(), index=LABELS)

,000,01,03,06,10,12,143,15,16,17,...,écoutes,éloges,épargne,époque,équipe,étais,était,étapes,être,última
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [291]:
# nltk.download('stopwords')

STOPWORDS = stopwords.words('english')

print(STOPWORDS)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [292]:
vectorizer = CountVectorizer(stop_words=STOPWORDS)

vectors = vectorizer.fit_transform(CLEAN_CORPUS)

pd.DataFrame(vectors.todense(), columns=vectorizer.get_feature_names(), index=LABELS)

/home/garima/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000,01,03,06,10,12,143,15,16,17,...,écoutes,éloges,épargne,époque,équipe,étais,était,étapes,être,última
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ed sheeran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rihanna,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [319]:
vectorizer = TfidfVectorizer(stop_words=STOPWORDS)

In [320]:
X = vectorizer.fit_transform(CLEAN_CORPUS)

In [321]:
lr = LogisticRegression()

In [322]:
lr.fit(X, LABELS)

LogisticRegression()

In [323]:
lr.score(X, LABELS)

0.9747899159663865

In [324]:
steps = [('tf-idf', TfidfVectorizer(stop_words=STOPWORDS)),
         
          ('LR', LogisticRegression())
        ]

pipeline = Pipeline(steps)

In [325]:
pipeline.fit(CLEAN_CORPUS, LABELS)

Pipeline(steps=[('tf-idf',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('LR', LogisticRegression())])

In [326]:
pipeline.predict(["The truth by my eye is my family."])

array(['ed sheeran'], dtype='<U10')

In [327]:
pipeline.predict(["love", "I want it that way"])

array(['ed sheeran', 'rihanna'], dtype='<U10')

In [328]:
pipeline.predict(["Lighters up, lighters up One time, lighters up Pulled up in the party When you saw me I was lighting up my J So go ahead and brighten up my day Lighters in the air when you're lighting up the rave"])

array(['ed sheeran'], dtype='<U10')

In [329]:
pipeline.predict(["Own it, girl, you just own it 'Cause your body's on fire Show me how to control it And go hit your spliff and get higher Girl, I love how you roll it I put my hand there, hold it"])

array(['rihanna'], dtype='<U10')

In [330]:
pipeline.predict(["Own it, you just own it 'Cause your body's on fire Show me how to control it And go hit your spliff and get higher, I love how you roll it I put my hand there, hold it"])

array(['ed sheeran'], dtype='<U10')

In [371]:
pipeline.predict(["The club isn't the best place to find a lover So the bar is where I go Me and my friends at the table doing shots Drinking fast and then we talk slow And you come over and start up a conversation with just me And trust me I'll give it a chance nowTake my hand, stop, put Van the Man on the jukebox And then we start to dance, and now I'm singing like"])

array(['ed sheeran'], dtype='<U10')

In [372]:
pipeline.predict(["I just hit the lick, south of France Currency exchange, lookin' fancy If that money dirty, make it dance 25 karats cost a mansion"])

array(['rihanna'], dtype='<U10')

### Naive Bayes

In [331]:
y_true = LABELS

In [332]:
def fit_model(X_train, y_train):
    """Return a trained model based on the corpus.
    Take the corpus and labels."""

    model = make_pipeline(
        TfidfVectorizer(),
        MultinomialNB()
    )

    fitted_model = model.fit(X_train, y_train)

    return fitted_model

In [333]:
def make_predictions(model, X_test):
    """Returns the class with most probability and also correspondent probabilities.
    Takes a fitted model and X_test (unseen data)."""

    X_test = [X_test]
    
    predictions = model.predict(X_test)
    probs = model.predict_proba(X_test)
    

    return predictions, probs

In [334]:
X_test_nb = "The truth by my eye is my family."

In [335]:
fitted_model_nb = fit_model(X_train = CORPUS, y_train = y_true)

In [336]:
predictions_nb = make_predictions(model = fitted_model, X_test = X_test_nb)
predictions_nb

(array(['ed sheeran'], dtype='<U10'), array([[0.71224681, 0.28775319]]))

In [337]:
import pickle

In [378]:
with open("NaiveBayes_nb.bin", "wb") as file:
    pickle.dump(fitted_model_nb, file)

In [379]:
X_test_nb_2 = "Own it, you just own it 'Cause your body's on fire Show me how to control it And go hit your spliff and get higher, I love how you roll it I put my hand there, hold it"

In [380]:
predictions_nb = make_predictions(model = fitted_model_nb, X_test = X_test_nb_2)
predictions_nb

(array(['rihanna'], dtype='<U10'), array([[0.47025269, 0.52974731]]))

In [381]:
X_test_nb_3 = "The club isn't the best place to find a lover So the bar is where I go Me and my friends at the table doing shots Drinking fast and then we talk slow And you come over and start up a conversation with just me And trust me I'll give it a chance nowTake my hand, stop, put Van the Man on the jukebox And then we start to dance, and now I'm singing like"

In [383]:
predictions_nb = make_predictions(model = fitted_model_nb, X_test = X_test_nb_3)
predictions_nb

(array(['ed sheeran'], dtype='<U10'), array([[0.66493368, 0.33506632]]))